In [1]:
import requests
import json

import os
import base64

In [2]:
def elastic_request(data=None, method=None, url=None):
    if method == None:
        method = requests.get
    if data:
        data = json.dumps(data)    

    return method(f"https://192.168.1.153:9200/{url}",
                  headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json",
                    "Authorization": f"ApiKey {os.getenv('ELASTIC_API_KEY')}"
                  },
                  verify=False,
                  data=data)

In [27]:
response = elastic_request(url="players-handbook-embedded/_search",
                           data={
                                "query": {
                                    "match": {
                                        "attachment.content": {
                                            "query": "Hunter's Mark",
                                            "boost": 0.7
                                        }
                                    }
                                },
                                "knn": {
                                    "field": "content-embedding",
                                    "k": 10,
                                    "boost": 0.3,
                                    "num_candidates": 10,
                                    "query_vector_builder": {
                                        "text_embedding": { 
                                            "model_id": "open-ai-embeddings", 
                                            "model_text": "Can I cast Hunter's Mark on an enemy who is invisible?" 
                                        }
                                    }
                                }
                           })

/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.153'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [28]:
hits = response.json()["hits"]["hits"]

In [29]:
[h["_source"]["attachment"]["content"] for h in hits]

["APPENDIX E: INSPIRATIONAL READING\n\nCredits\n\nD&D Lead Designers: Mike Mearls, Jeremy Crawford\n\nPlayer’s Handbook Lead: Jeremy Crawford\n\nRules Development: Rodney Thompson, Peter Lee\n\nWriting: James Wyatt, Robert J. Schwalb, Bruce R. Cordell\n\nEditing: Michele Carter, Chris Sims, Scott Fitzgerald Gray, Christopher\n\nPerkins\n\nProducer: Greg Bilsland\n\nArt Directors: Kate Irwin, Dan Gelon, Jon Schindehette, Mari Kolkowsky,\n\nMelissa Rapier, Shauna Narciso\n\nGraphic Designers: Bree Heiss, Emi Tanji, Barry Craig\n\nCover Illustrator: Tyler Jacobson\n\nInterior Illustrators: Steve Argyle, Tom Babbey, Daren Bader, Drew Baker,\n\nMark Behm, Eric Belisle, Christopher Bradley, Noah Bradley, Sam\n\nBurley, Clint Cearley, Milivoj ´Ceran, Sidharth Chaturvedi, Jedd\n\nChevrier, jD, Allen Douglas, Jesper Ejsing, Craig Elliott, Wayne\n\nEngland, Scott M. Fischer, Randy Gallegos, Justin Gerard, Florian De\n\nGesincourt, Lars Grant-West, Jon Hodgson, Ralph Horsley, Lake\n\nHurwitz, Tyl